In [2]:
!sudo apt update
!sudo apt install libcairo2-dev \
    texlive texlive-latex-extra texlive-fonts-extra \
    texlive-latex-recommended texlive-science \
    tipa libpango1.0-dev
!pip install manim
!pip install IPython==8.21.0
from manim import *
import numpy as np

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,369 kB]
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,730 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/

Escolha o número que quer aproximar a raiz, e quantas casas gostaria de aproximar

In [2]:
numero = 17
casas = 4

In [1]:
%%manim -qm -v WARNING Sqrt2Approximation

class Sqrt2Approximation(ZoomedScene):
    def create_zoomed_line(self, start, end, interval_size, step_size, level):
        """
        Creates a zoomed-in number line with appropriate labels and scaling.
        """
        zoomed_line = NumberLine(
            x_range=[start - interval_size, end + interval_size, interval_size / 10],
            numbers_with_elongated_ticks=[start, end],
            length=30  # Adjust length for visibility
        )

        # Add labels to the zoomed line
        zoomed_line.add_labels(
            {start: f"{start:.{level}f}"} |
            {num: str(n + 1) for n, num in enumerate(np.linspace(start + step_size, end - step_size, 9))} |
            {end: f"{end:.{level}f}"}
        )

        # Center the zoomed line
        zoomed_line.shift(ORIGIN - zoomed_line.n2p((start + end) / 2))
        return zoomed_line

    def update_approximation(self, point, interval_size, level):
        """
        Updates the approximation value based on the zoom level.
        (Not used in the new digit-selection procedure.)
        """
        approx_val = np.round(np.floor(point / interval_size * 10) * interval_size / 10, level + 1)
        return approx_val

    def zoom_to_point(self, current_line, rect, point, level, grp, frame):
        """
        Zoom into a specific point on the number line, display all 10 candidate
        next approximations (and their squares) in a 5x2 grid, highlight the candidates
        whose squares are just below and just above 2, highlight these on the number line,
        and then proceed with the zoom.
        """
        # Calculate interval and step size
        interval_size = 10 ** -(level-1)
        start = np.round(np.floor(point / interval_size) * interval_size, level)
        end = np.round(start + interval_size, level)
        step_size = interval_size / 10

        # Create the zoomed line
        zoomed_line = self.create_zoomed_line(start, end, interval_size, step_size, level)

        # Apply homotopy effect for zoom animation
        if level != 0:
            displacement = (current_line.p2n(ORIGIN) - np.mean([start, end])) * 10 ** (level)
            self.play(
                Homotopy(
                    lambda x, y, z, t: [
                        (1 - t) * x + t * (10 * x + displacement),
                        (1 - t) * y + t * 10 * y,
                        z
                    ],
                    VGroup(current_line, rect)
                )
            )
            self.wait(2)  # Pause to emphasize the zoom

        # Display the zoomed line and fade out the previous line
        self.play(Write(zoomed_line, run_time=2), FadeOut(current_line, run_time=0.5))
        self.wait(2)

        # Build the 10 candidate approximations and their squares.
        candidates = [start + i * step_size for i in range(10)]
        squares = [cand ** 2 for cand in candidates]

        # Create a grid of MathTex objects for each candidate showing its square.
        # We use 2 rows and 5 columns.
        grid_items = VGroup(*[
            MathTex("(",f"{cand:.{level}f}",f")^2 = {cand**2:.{2*level}f}").scale(.8)
            for cand in candidates
        ])
        grid_items.arrange_in_grid(rows=5, cols=2, buff=(1, .1), flow_order='dr')
        grid_items.move_to(np.array([0, -self.camera.frame.get_height()/2 + self.camera.frame.get_height()/4, 0]))


        # Write the grid on screen.
        self.play([TransformFromCopy(zoomed_line.labels[i], grid_items[i]) for i in range(10)])
        self.wait(2)

        # Find the candidate just below (largest with square <=2)
        # and the candidate just above (smallest with square >2).
        below_candidates = [(cand, sq) for cand, sq in zip(candidates, squares) if sq <= numero]
        above_candidates = [(cand, sq) for cand, sq in zip(candidates, squares) if sq > numero]
        if below_candidates:
            below_cand = below_candidates[-1][0]
        else:
            below_cand = candidates[0]
        if above_candidates:
            above_cand = above_candidates[0][0]
        else:
            above_cand = candidates[-1]

        # Highlight the corresponding grid items.
        below_index = candidates.index(below_cand)
        above_index = candidates.index(above_cand)
        self.play(
            Flash(grid_items[below_index], color=YELLOW, flash_radius=grid_items[below_index].get_width() / 2),
            grid_items[below_index].animate.set_color(GREEN),
            Flash(grid_items[above_index], color=YELLOW, flash_radius=grid_items[above_index].get_width() / 2),
            grid_items[above_index].animate.set_color(RED)
        )
        self.wait(1)

        # Also highlight these candidates on the number line.
        # (The number line labels were created with keys matching candidate values.)
        self.play(
            Transform(grid_items[below_index], zoomed_line.labels[below_index].set_color(GREEN)),
            Transform(grid_items[above_index], zoomed_line.labels[above_index].set_color(RED)),
        )
        self.wait(1)

        # Choose the candidate just below 2 as the new approximation.
        new_approx = below_cand

        # Update the approximation display.
        approx_text = MathTex(f"{new_approx:.{level}f}")
        text_label = Tex("Aproximação: ").next_to(approx_text, LEFT)
        new_grp = VGroup(approx_text, text_label).add_updater(lambda m: m.align_to(frame, UR))

        # Fade out the grid, update the line and the approximation text.
        self.play(
            FadeOut(grid_items),
            Transform(grp, new_grp)
        )
        self.wait(2)

        return zoomed_line, new_approx

    def construct(self):
        """
        Main construction of the scene, including the initial setup and successive zoom-ins.
        """
        # Create the initial number line.
        initial_line = NumberLine(
            x_range=[0, 10, 1],
            include_numbers=True,
            numbers_with_elongated_ticks=[0, 10]
        )
        initial_line.shift(ORIGIN - initial_line.n2p(5)).scale(0)

        # Create the (invisible) rectangle.
        rect = Rectangle(
            width=np.sqrt(2),
            height=1,
            fill_opacity=0,
            stroke_width=0
        ).align_to(initial_line.n2p(0), LEFT + DOWN).add_updater(
            lambda m: m.set_opacity(0).set_stroke_opacity(0)
        )

        # Create initial approximation text.
        approx_text = MathTex()
        text_label = Tex().next_to(approx_text, LEFT)

        # Add the frame for alignment.
        frame = Rectangle().add_updater(
            lambda m: m.move_to(self.camera.frame_center)
            .stretch_to_fit_width(self.camera.frame_width * 0.9)
            .stretch_to_fit_height(self.camera.frame_height * 0.9)
            .set_opacity(0)
        )
        self.add(frame)
        grp = VGroup(approx_text, text_label).add_updater(lambda m: m.align_to(frame, UR))

        # Successive zoom-ins.
        zoomed_line = initial_line
        point = np.sqrt(numero)  # Constant target value.
        decimal_places = casas

        title = MathTex(point).align_to(frame, UP)

        for level in range(decimal_places+1):  # Perform 2 levels of zoom (can increase as desired)
            zoomed_line, approx_val = self.zoom_to_point(zoomed_line, rect, point, level, grp, frame)


UsageError: Cell magic `%%manim` not found.
